In [17]:
import numpy as np
import os
import pandas as pd
import time
import random
import pickle
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import PIL
# Import PuLP modeler functions
from pulp import *

In [18]:
cwd = os.getcwd()
plt.close('all')

# Set random seed
random.seed(42)



In [19]:
df = pd.read_excel(os.path.join(cwd,'Knight_network.xlsx'), keep_default_na=False)

In [20]:
df.head()

,Origin,Destination,Name,Cost
0,A,B,Innocuous bridge over water,3
1,A,C,Cat warrior,8
2,A,D,Angry ogre,12
3,B,E,Stereotypical witch,10
4,B,F,Spooky ghost,9


In [21]:
# Define unique set of nodes
Nodes = sorted(list(set(df.Origin) | set(df.Destination)))
N     = {Nodes[idx]:idx for idx,n in enumerate(Nodes)}

# Define edges (we have one distinct edge per row of the input dataframe)
E = {}
for index, row in df.iterrows():
    E[row.Origin,row.Destination] = [row.Origin,row.Destination,row.Name,row.Cost]
    
# Define sets of ingoing/outgoing edges per node

N_in  = {node:[E[key][0] for key in E.keys() if E[key][1]==node] for node in N.keys()}
N_out = {node:[E[key][1] for key in E.keys() if E[key][0]==node] for node in N.keys()}

In [22]:
N_in

{'A': [],
 'B': ['A'],
 'C': ['A'],
 'D': ['A', 'C'],
 'E': ['B'],
 'F': ['B', 'C', 'D', 'E'],
 'G': ['E'],
 'H': ['E', 'F', 'I'],
 'I': ['D', 'F'],
 'J': ['D'],
 'K': ['E', 'G', 'H'],
 'L': ['I', 'J'],
 'M': ['H', 'K', 'L']}

In [23]:
# Define source and sink nodes
source = 'A'
sink   = 'M'

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Shortest_path", LpMinimize)

In [24]:
x_ij_vars = LpVariable.dicts("x",[(E[key][0],E[key][1]) for key in E.keys()],0,1)

In [25]:
x_ij_vars

{('A', 'B'): x_('A',_'B'),
 ('A', 'C'): x_('A',_'C'),
 ('A', 'D'): x_('A',_'D'),
 ('B', 'E'): x_('B',_'E'),
 ('B', 'F'): x_('B',_'F'),
 ('C', 'D'): x_('C',_'D'),
 ('C', 'F'): x_('C',_'F'),
 ('D', 'F'): x_('D',_'F'),
 ('D', 'I'): x_('D',_'I'),
 ('D', 'J'): x_('D',_'J'),
 ('E', 'F'): x_('E',_'F'),
 ('E', 'G'): x_('E',_'G'),
 ('E', 'H'): x_('E',_'H'),
 ('E', 'K'): x_('E',_'K'),
 ('F', 'H'): x_('F',_'H'),
 ('F', 'I'): x_('F',_'I'),
 ('G', 'K'): x_('G',_'K'),
 ('H', 'K'): x_('H',_'K'),
 ('H', 'M'): x_('H',_'M'),
 ('I', 'H'): x_('I',_'H'),
 ('I', 'L'): x_('I',_'L'),
 ('J', 'L'): x_('J',_'L'),
 ('K', 'M'): x_('K',_'M'),
 ('L', 'M'): x_('L',_'M')}

In [26]:
E

{('A', 'B'): ['A', 'B', 'Innocuous bridge over water', 3],
 ('A', 'C'): ['A', 'C', 'Cat warrior', 8],
 ('A', 'D'): ['A', 'D', 'Angry ogre', 12],
 ('B', 'E'): ['B', 'E', 'Stereotypical witch', 10],
 ('B', 'F'): ['B', 'F', 'Spooky ghost', 9],
 ('C', 'D'): ['C', 'D', 'Friendly wizard', 1],
 ('C', 'F'): ['C', 'F', 'Dangerous bridge', 5],
 ('D', 'F'): ['D', 'F', 'Stroll in the woods', 3],
 ('D', 'I'): ['D', 'I', 'Infested swamp', 15],
 ('D', 'J'): ['D', 'J', 'Regular bridge', 2],
 ('E', 'F'): ['E', 'F', 'Friend of the angry ogre', 12],
 ('E', 'G'): ['E', 'G', 'Bulky knight', 9],
 ('E', 'H'): ['E', 'H', 'Calm forest', 4],
 ('E', 'K'): ['E', 'K', 'Three-headed dragon', 11],
 ('F', 'H'): ['F', 'H', 'Tiny dragon', 6],
 ('F', 'I'): ['F', 'I', 'Merlin-type wizard', 1],
 ('G', 'K'): ['G', 'K', 'Mysterious cave', 7],
 ('H', 'K'): ['H', 'K', 'Dark archeress', 8],
 ('H', 'M'): ['H', 'M', 'Chained ghost', 13],
 ('I', 'H'): ['I', 'H', 'Path in the woods', 4],
 ('I', 'L'): ['I', 'L', 'You shall not pass

In [27]:
# The objective function is added to 'prob' first
prob += (
    lpSum([E[key][3] * x_ij_vars[E[key][0],E[key][1]] for idx,key in enumerate(E.keys())]),
    "Total_distance",
)

In [28]:
print('Creating constraints')

for n in N.keys():
    if n == source:
        prob += (
            lpSum(x_ij_vars[(n,n_out)] for n_out in N_out[n]) -
            lpSum(x_ij_vars[(n_in,n)] for n_in in N_in[n]) == 1,
            "Source",)
    elif n == sink:
        prob += (
            lpSum(x_ij_vars[(n,n_out)] for n_out in N_out[n]) -
            lpSum(x_ij_vars[(n_in,n)] for n_in in N_in[n]) == -1,
            "Sink",)
    else:
        prob += (
            lpSum(x_ij_vars[(n,n_out)] for n_out in N_out[n]) -
            lpSum(x_ij_vars[(n_in,n)] for n_in in N_in[n]) == 0,
            "node_%s"%(n),)

Creating constraints


In [29]:
# The problem data is written to an .lp file
prob.writeLP("SP.lp")

# The problem is solved using PuLP's choice of Solver
solver = getSolver('GLPK_CMD')
prob.solve()

1

In [30]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Length path = ", value(prob.objective))

Status: Optimal
x_('A',_'B') = 0.0
x_('A',_'C') = 1.0
x_('A',_'D') = 0.0
x_('B',_'E') = 0.0
x_('B',_'F') = -0.0
x_('C',_'D') = 1.0
x_('C',_'F') = 0.0
x_('D',_'F') = 0.0
x_('D',_'I') = 0.0
x_('D',_'J') = 1.0
x_('E',_'F') = 0.0
x_('E',_'G') = 0.0
x_('E',_'H') = 0.0
x_('E',_'K') = 0.0
x_('F',_'H') = 0.0
x_('F',_'I') = 0.0
x_('G',_'K') = 0.0
x_('H',_'K') = 0.0
x_('H',_'M') = 0.0
x_('I',_'H') = 0.0
x_('I',_'L') = 0.0
x_('J',_'L') = 1.0
x_('K',_'M') = -0.0
x_('L',_'M') = 1.0
Length path =  22.0


In [31]:
# The optimised objective function value is printed to the screen
print("Length path = ", value(prob.objective))

Length path =  22.0
